Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [3]:
import tensorflow as tf
from PIL import Image
from torch.utils.data import Dataset
import os
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [9]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 100

In [11]:
# Load in data
train_labels = pd.read_csv("data/train_labels.csv")
test_labels = pd.read_csv("data/test_labels.csv")

# change class labels from str to int
name_to_int_dict = {'ace of spades': 0, 'two of spades':1, 'three of spades':2, 'four of spades':3, 
                    'five of spades':4, 'six of spades':5, 'seven of spades':6, 'eight of spades':7, 
                    'nine of spades':8, 'ten of spades':9, 'jack of spades':10, 'queen of spades':11, 
                    'king of spades':12, 'ace of hearts': 13, 'two of hearts': 14, 'three of hearts': 15, 
                    'four of hearts': 16, 'five of hearts': 17, 'six of hearts': 18, 'seven of hearts':19, 
                    'eight of hearts':20, 'nine of hearts':21, 'ten of hearts':22, 'jack of hearts': 23, 
                    'queen of hearts': 24, 'king of hearts':25, 'ace of clubs': 26, 'two of clubs': 27, 
                    'three of clubs': 28, 'four of clubs': 29, 'five of clubs': 30, 'six of clubs': 31,
                    'seven of clubs':32, 'eight of clubs':33, 'nine of clubs':34, 'ten of clubs':35, 
                    'jack of clubs': 36, 'queen of clubs': 37, 'king of clubs':38, 'ace of diamonds': 39, 
                    'two of diamonds': 40, 'three of diamonds': 41, 'four of diamonds': 42, 'five of diamonds': 43, 
                    'six of diamonds': 44,'seven of diamonds':45, 'eight of diamonds':46, 'nine of diamonds':47, 
                    'ten of diamonds':48, 'jack of diamonds': 49, 'queen of diamonds': 50,'king of diamonds':51,
                    
                    # these are to account for some typos in the dataset
                    'three of dimaonds': 41, 'four of dimaonds': 42, 'five of dimaonds': 43, 'six of dimaonds': 44, 
                    'eigth of clubs':33, 'seven of seven':32 
                    }

test_labels['class_number'] = test_labels['class'].map(name_to_int_dict)
train_labels['class_number'] = train_labels['class'].map(name_to_int_dict)

# These should print empty to show that all labels are accounted for
print(test_labels.loc[test_labels['class_number'].isnull()])
print(train_labels.loc[train_labels['class_number'].isnull()])

image_paths_train = train_labels['filename'].values
labels_train = train_labels['class_number'].values

image_paths_test = test_labels['filename'].values
labels_test = test_labels['class_number'].values

Empty DataFrame
Columns: [filename, width, height, class, xmin, ymin, xmax, ymax, class_number]
Index: []
Empty DataFrame
Columns: [filename, width, height, class, xmin, ymin, xmax, ymax, class_number]
Index: []


(539, 539, 98, 98)

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_paths_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((image_paths_test, labels_test))

def load_train_image(image_path, label):
    image_path = tf.strings.join(['train/', image_path], separator='')
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def load_test_image(image_path, label):
    image_path = tf.strings.join(['test/', image_path], separator='')
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_test_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [17]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        self.architecture = [              
              tf.keras.layers.Conv2D(100, (5, 5), strides=(2,2)),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.Dropout(0.4),
              tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
              
              tf.keras.layers.Conv2D(250, (4, 4), strides=(1,1)),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.Dropout(0.4),
              tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
              
              tf.keras.layers.Conv2D(450, (3, 3), strides=(1,1)),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
              
              tf.keras.layers.Conv2D(650, (3, 3), strides=(1,1)),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.Conv2D(650, (3, 3), strides=(1,1)),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
              
              #Flatten
              tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
              
              tf.keras.layers.Dense(300),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.Dropout(0.6),
              
              tf.keras.layers.Dense(300),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.Dropout(0.6),
              
              tf.keras.layers.Dense(200),
              tf.keras.layers.BatchNormalization(),
              tf.keras.layers.ReLU(),
              tf.keras.layers.Dropout(0.6),
              
              tf.keras.layers.Dense(15, activation='softmax')]
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

In [30]:
model = CardPredictor()
model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])
model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 11s 384ms/step - accuracy: 0.0264 - loss: 0.7984 - val_accuracy: 0.0204 - val_loss: 1.0335
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 349ms/step - accuracy: 0.0121 - loss: 1.2360 - val_accuracy: 0.0102 - val_loss: 9.9040
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 372ms/step - accuracy: 0.0170 - loss: 29.7570 - val_accuracy: 0.0204 - val_loss: 188.5230
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 377ms/step - accuracy: 0.0160 - loss: 404.9336 - val_accuracy: 0.0102 - val_loss: 1231.3318
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 383ms/step - accuracy: 0.0096 - loss: 3525.3657 - val_accuracy: 0.0204 - val_loss: 12701.6865
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 396ms/step - accuracy: 0.0148 - loss: 19718.2598 - val_accuracy: 0.0102 - val_loss: 79958.2188
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 384ms/step - accuracy: 0.0153 - loss: 91445.2422 - val_accuracy: 0.0306 - val_loss: 300491.5625
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 376ms/step - accuracy:

KeyboardInterrupt: 